In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install torch transformers datasets peft bitsandbytes trl accelerate pandas nltk tf-keras -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00


In [3]:
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

def download(url):
    try:
        file_id = url.split('/d/')[1].split('/')[0]
        download_url = f'https://drive.google.com/uc?id={file_id}'

        response = requests.get(download_url)
        response.raise_for_status()

        return response.content

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        return None

url = "https://drive.google.com/file/d/14kplogWzU2JsIB04ENeLtgtYTt0kYyxU"
file_content = download(url)


if file_content:
    try:
      with ZipFile(BytesIO(file_content)) as zip_ref:
          zip_ref.extractall()
          print("Zip file extracted successfully!")
    except BadZipFile:
      print("Invalid zip file.")
    except Exception as e:
      print(f"An error occurred: {e}")

Zip file extracted successfully!


In [20]:
import pandas as pd
df = pd.read_csv('inference_dataset/train.tsv', sep='\t')
df = df.rename(columns={'Sent1_parse':'premise', "Sent2_parse":'hypothesis', 'Label':'label'})
df.head()

,premise,hypothesis,label
0,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,neutral
1,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,contradiction
2,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,entailment
3,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,neutral
4,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,entailment


In [21]:
import re
import nltk

def extract_raw_text(pcfg_text):
    """Parses the PCFG tree using NLTK and extracts only the words."""
    try:
        tree = nltk.Tree.fromstring(pcfg_text)  # Convert PCFG string to tree
        return " ".join(tree.leaves())  # Extract leaf nodes (words)
    except Exception as e:
        return None

df['premise'] = df['premise'].apply(extract_raw_text)
df['hypothesis'] = df['hypothesis'].apply(extract_raw_text)
df.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competiti...,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner , ordering an omelette .",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors , on a horse .",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment


In [6]:
# label2id = {"entailment": 0, "contradiction": 1, "neutral": 2}
# df["label"] = df["label"].map(label2id)

In [32]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!



```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

In [11]:
!pip install -q datasets

In [ ]:
from datasets import Dataset
from unsloth.chat_templates import standardize_sharegpt

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
)

def convert_to_sharegpt(row):
    return {
        "conversations": [
            {
                "from": "system",
                "value": """You are an expert in Natural Language Inference (NLI).
                    Your task is to analyze a premise and a hypothesis, then determine the logical relationship between them.
                    Choose only one label: 'entailment', 'contradiction', or 'neutral'."""

            },
            {
                "from": "user",
                "value": f"""Premise: {row['premise']}\n
                    Hypothesis: {row['hypothesis']}\n
                    Does the hypothesis logically follow from the premise?
                    Answer with 'entailment', 'contradiction', or 'neutral'."""

            },
            {
                "from": "assistant",
                "value":row['label']
            },
        ]
    }

dataset = Dataset.from_pandas(df)

dataset = dataset.map(convert_to_sharegpt)

dataset = standardize_sharegpt(dataset)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/550152 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/550152 [00:00<?, ? examples/s]

In [42]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=2048,  # Adjust based on your model
    )

dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/550152 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [34]:
dataset[0]

{'premise': 'A person on a horse jumps over a broken down airplane .',
 'hypothesis': 'A person is training his horse for a competition .',
 'label': 'neutral',
 'conversations': [{'content': "You are an expert in Natural Language Inference (NLI).\n                    Your task is to analyze a premise and a hypothesis, then determine the logical relationship between them.\n                    Choose only one label: 'entailment', 'contradiction', or 'neutral'.",
   'role': 'system'},
  {'content': "Premise: A person on a horse jumps over a broken down airplane .\n\n                    Hypothesis: A person is training his horse for a competition .\n\n                    Does the hypothesis logically follow from the premise?\n                    Answer with 'entailment', 'contradiction', or 'neutral'.",
   'role': 'user'},
  {'content': 'neutral', 'role': 'assistant'}],
 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Da

In [35]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [36]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 550,152 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [40]:
dataset[0]

{'premise': 'A person on a horse jumps over a broken down airplane .',
 'hypothesis': 'A person is training his horse for a competition .',
 'label': 'neutral',
 'conversations': [{'content': "You are an expert in Natural Language Inference (NLI).\n                    Your task is to analyze a premise and a hypothesis, then determine the logical relationship between them.\n                    Choose only one label: 'entailment', 'contradiction', or 'neutral'.",
   'role': 'system'},
  {'content': "Premise: A person on a horse jumps over a broken down airplane .\n\n                    Hypothesis: A person is training his horse for a competition .\n\n                    Does the hypothesis logically follow from the premise?\n                    Answer with 'entailment', 'contradiction', or 'neutral'.",
   'role': 'user'},
  {'content': 'neutral', 'role': 'assistant'}],
 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Da